# Producer

The data was generated based on the following formula:

$$
y(t+1) = -0.4 \frac{(3 - y(t)^2)}{(1 + y(t)^2)} + 0.6 \frac{(3 - (y(t-1) - 0.5)^3)}{(1 + (y(t-1) - 0.5)^3)}
$$


In [1]:
from kafka import KafkaProducer
import time
import numpy as np
import json
from message import Message

In [2]:
producer = KafkaProducer(bootstrap_servers='localhost:9092')

In [3]:
def get_y_future(y_current, y_before):
    """
    Calculates the y(t+1) based on y(t) and y(t-1)
    
    Parameters
    ----------
    arg1: float
        represents y(t)
    arg2: float
        represents y(t-1)
        
    Returns
    -------
    float
        y(t+1)
    """
    return -0.4 * (3 - pow(y_current, 2))/(1 + pow(y_current, 2)) + 0.6 * (3 - pow(y_before - 0.5, 3)) / (1 + pow(y_before - 0.5, 3))


In [4]:
def send_message(message, producer, topic_name):
    """
    It uses the producer to sent a message using that topic's name
    
    Parameters
    ----------
    arg1: Message
        an object containing the id and the inputs xs
    arg2: KafkaProducer
    arg3: string
    
    Returns
    -------
    string
        JSON message that was sent
    """

    message_json = json.dumps(message.__dict__)
    producer.send(topic_name, message_json.encode())
    return message_json

In [8]:
np.random.seed(2452020515) # Fix seed to ensure repeatability

time_delay = 5
messages_sent = 0

T = 1000
stream_size = 100 
topic_name = "timeSerieS"

# y(t-1), y(t), y(t+1)
y_before = np.random.random()
y_current = np.random.random()
y_future = None

message = Message()

#Infinite loop for sending messages to Kafka with the topic_name
while T > 0:
    message.x = []
    message.id += 1
    message.x.append(y_before)
    message.x.append(y_current)
    
    current_stream_size = min(T, stream_size)
    
    for i in range(current_stream_size - 2):
        y_future = get_y_future(y_current, y_before)
        message.x.append(y_future)
        
        y_before = y_current
        y_current = y_future
        
    print('\t ID ', message.id,' SIZE ', current_stream_size, ' INPUT ', message.x[:5])
    print('\r\n')
    
    send_message(message, producer, topic_name)
    time.sleep(time_delay)
        
    T -= stream_size
    

	 ID  1  SIZE  100  INPUT  [0.0024151530439742164, 0.027939322560110558, 0.938466063764634, 1.631408777013852, 1.5764373213527065]


	 ID  2  SIZE  100  INPUT  [-1.2807545908243094, -1.0525911914735415, -1.4755022113204022, -1.5786905435719916, -1.0158512925006284]


	 ID  3  SIZE  100  INPUT  [-1.0635969823606095, -1.3764500615613278, -1.6029890634728405, -1.0762613380250907, -1.2304466463965211]


	 ID  4  SIZE  100  INPUT  [-1.256104043220387, -1.6398974594840607, -1.1772120022268324, -1.143386963955084, -1.538938195958925]


	 ID  5  SIZE  100  INPUT  [-1.5441783616434201, -1.3558942012725201, -1.0819087504225262, -1.3822243844384996, -1.560913189195222]


	 ID  6  SIZE  100  INPUT  [-1.5530948614797917, -1.08900788017766, -1.245504416706639, -1.6239065209930805, -1.1957042431657583]


	 ID  7  SIZE  100  INPUT  [-1.1938413474335727, -1.139870195374434, -1.5176569771892008, -1.388200292348849, -1.0793136136334378]


	 ID  8  SIZE  100  INPUT  [-1.0797760400803897, -1.35888661543787